In [14]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from time import sleep
import concurrent.futures
import csv

In [15]:
# 東京大学技術経営戦略学専攻の進路
companies = [
    "ゴールドマン・サックス証券", "JPモルガン証券", "メリルリンチ日本証券", "大和証券キャピタル・マーケッツ",
    "東京海上日動火災保険", "日本政策投資銀行", "UBS証券株式会社", "野村證券", "三菱UFJ銀行", "三井住友信託銀行",
    "A.T.カーニー", "ボストン・コンサルティンググループ", "マッキンゼー・アンド・カンパニー・インク・ジャパン", "三菱総合研究所",
    "サントリーホールディングス", "ソニーグループ株式会社", "株式会社日立製作所", "プロクター・アンド・ギャンブル・ジャパン",
    "日本アイ・ビー・エム", "楽天グループ株式会社",
    "三菱商事株式会社", "三井物産株式会社", "三井不動産株式会社", "ソフトバンクグループ"
]

# 日経225構成銘柄
companies = [
    "協和キリン（株）", "武田薬品工業（株）", "アステラス製薬（株）", "住友ファーマ（株）", "塩野義製薬（株）", 
    "中外製薬（株）", "エーザイ（株）", "第一三共（株）", "大塚ホールディングス（株）",
    "ミネベアミツミ（株）", "（株）日立製作所", "三菱電機（株）", "富士電機（株）", "（株）安川電機", 
    "ニデック（株）", "オムロン（株）", "（株）ジーエス・ユアサ コーポレーション", "日本電気（株）", 
    "富士通（株）", "ルネサスエレクトロニクス（株）", "セイコーエプソン（株）", "パナソニック ホールディングス（株）", 
    "シャープ（株）", "ソニーグループ（株）", "ＴＤＫ（株）", "アルプスアルパイン（株）", "横河電機（株）", 
    "（株）アドバンテスト", "（株）キーエンス", "（株）デンソー", "レーザーテック（株）", "カシオ計算機（株）", 
    "ファナック（株）", "京セラ（株）", "太陽誘電（株）", "（株）村田製作所", "（株）ＳＣＲＥＥＮホールディングス", 
    "キヤノン（株）", "（株）リコー", "東京エレクトロン（株）", "日産自動車（株）", "いすゞ自動車（株）", 
    "トヨタ自動車（株）", "日野自動車（株）", "三菱自動車工業（株）", "マツダ（株）", "本田技研工業（株）", 
    "スズキ（株）", "（株）ＳＵＢＡＲＵ", "ヤマハ発動機（株）", "テルモ（株）", "コニカミノルタ（株）", 
    "（株）ニコン", "オリンパス（株）", "ＨＯＹＡ（株）", "シチズン時計（株）", "日本電信電話（株）", 
    "ＫＤＤＩ（株）", "ソフトバンク（株）", "（株）ＮＴＴデータグループ", "ソフトバンクグループ（株）", "（株）しずおかフィナンシャルグループ", 
    "（株）コンコルディア・フィナンシャルグループ", "（株）あおぞら銀行", "（株）三菱ＵＦＪフィナンシャル・グループ", "（株）りそなホールディングス", 
    "三井住友トラスト・ホールディングス（株）", "（株）三井住友フィナンシャルグループ", "（株）千葉銀行", "（株）ふくおかフィナンシャルグループ", 
    "（株）みずほフィナンシャルグループ", "（株）クレディセゾン", "オリックス（株）", "（株）日本取引所グループ", "（株）大和証券グループ本社", 
    "野村ホールディングス（株）", "ＳＯＭＰＯホールディングス（株）", "ＭＳ＆ＡＤインシュアランスグループホールディングス（株）", "第一生命ホールディングス（株）", 
    "東京海上ホールディングス（株）", "（株）Ｔ＆Ｄホールディングス", "（株）ニッスイ", "（株）日清製粉グループ本社", "明治ホールディングス（株）", 
    "日本ハム（株）", "サッポロホールディングス（株）", "アサヒグループホールディングス（株）", "キリンホールディングス（株）", "宝ホールディングス（株）", 
    "キッコーマン（株）", "味の素（株）", "（株）ニチレイ", "日本たばこ産業（株）", "Ｊ．フロント リテイリング（株）", 
    "（株）三越伊勢丹ホールディングス", "（株）セブン＆アイ・ホールディングス", "（株）高島屋", "（株）丸井グループ", "イオン（株）", 
    "（株）ニトリホールディングス", "（株）ファーストリテイリング", "エムスリー（株）", "（株）ディー・エヌ・エー", "（株）ネクソン", 
    "（株）電通グループ", "（株）メルカリ", "（株）オリエンタルランド", "ＬＩＮＥヤフー（株）", "トレンドマイクロ（株）", 
    "（株）サイバーエージェント", "楽天グループ（株）", "（株）リクルートホールディングス", "日本郵政（株）", "任天堂（株）", 
    "東宝（株）", "セコム（株）", "コナミグループ（株）", "（株）ＩＮＰＥＸ", "帝人（株）", 
    "東レ（株）", "王子ホールディングス（株）", "日本製紙（株）", "（株）クラレ", "旭化成（株）", 
    "（株）レゾナック・ホールディングス", "住友化学（株）", "日産化学（株）", "東ソー（株）", "（株）トクヤマ", 
    "デンカ（株）", "信越化学工業（株）", "三井化学（株）", "三菱ケミカルグループ（株）", "ＵＢＥ（株）", 
    "花王（株）", "ＤＩＣ（株）", "富士フイルムホールディングス（株）", "（株）資生堂", "日東電工（株）", 
    "出光興産（株）", "ＥＮＥＯＳホールディングス（株）", "横浜ゴム（株）", "（株）ブリヂストン", "ＡＧＣ（株）", 
    "日本電気硝子（株）", "住友大阪セメント（株）", "太平洋セメント（株）", "東海カーボン（株）", "ＴＯＴＯ（株）", 
    "日本碍子（株）", "日本製鋼所（株）", "オークマ（株）", "（株）アマダ", "ＳＭＣ（株）", 
    "（株）小松製作所", "住友重機械工業（株）", "日立建機（株）", "（株）クボタ", "（株）荏原製作所", 
    "ダイキン工業（株）", "日本精工（株）", "ＮＴＮ（株）", "（株）ジェイテクト", "日立造船（株）", 
    "三菱重工業（株）", "（株）ＩＨＩ", "川崎重工業（株）", "（株）バンダイナムコホールディングス", "ＴＯＰＰＡＮホールディングス（株）", 
    "大日本印刷（株）", "ヤマハ（株）", "東急不動産ホールディングス（株）", "三井不動産（株）", "三菱地所（株）", 
    "東京建物（株）", "住友不動産（株）", "東武鉄道（株）", "東急（株）", "小田急電鉄（株）", 
    "京王電鉄（株）", "京成電鉄（株）", "東日本旅客鉄道（株）", "西日本旅客鉄道（株）", "東海旅客鉄道（株）", 
    "ヤマトホールディングス（株）", "ＮＩＰＰＯＮ ＥＸＰＲＥＳＳホールディングス（株）", "日本郵船（株）", "（株）商船三井", "川崎汽船（株）", 
    "日本航空（株）", "ＡＮＡホールディングス（株）", "三菱倉庫（株）", "東京電力ホールディングス（株）", "中部電力（株）", 
    "関西電力（株）", "東京瓦斯（株）", "大阪瓦斯（株）"
]


# リスト内の各社名で "（株）" を "株式会社" に置換
companies = [company.replace("（株）", "株式会社") for company in companies]

# 結果を表示
print(companies)


['協和キリン株式会社', '武田薬品工業株式会社', 'アステラス製薬株式会社', '住友ファーマ株式会社', '塩野義製薬株式会社', '中外製薬株式会社', 'エーザイ株式会社', '第一三共株式会社', '大塚ホールディングス株式会社', 'ミネベアミツミ株式会社', '株式会社日立製作所', '三菱電機株式会社', '富士電機株式会社', '株式会社安川電機', 'ニデック株式会社', 'オムロン株式会社', '株式会社ジーエス・ユアサ コーポレーション', '日本電気株式会社', '富士通株式会社', 'ルネサスエレクトロニクス株式会社', 'セイコーエプソン株式会社', 'パナソニック ホールディングス株式会社', 'シャープ株式会社', 'ソニーグループ株式会社', 'ＴＤＫ株式会社', 'アルプスアルパイン株式会社', '横河電機株式会社', '株式会社アドバンテスト', '株式会社キーエンス', '株式会社デンソー', 'レーザーテック株式会社', 'カシオ計算機株式会社', 'ファナック株式会社', '京セラ株式会社', '太陽誘電株式会社', '株式会社村田製作所', '株式会社ＳＣＲＥＥＮホールディングス', 'キヤノン株式会社', '株式会社リコー', '東京エレクトロン株式会社', '日産自動車株式会社', 'いすゞ自動車株式会社', 'トヨタ自動車株式会社', '日野自動車株式会社', '三菱自動車工業株式会社', 'マツダ株式会社', '本田技研工業株式会社', 'スズキ株式会社', '株式会社ＳＵＢＡＲＵ', 'ヤマハ発動機株式会社', 'テルモ株式会社', 'コニカミノルタ株式会社', '株式会社ニコン', 'オリンパス株式会社', 'ＨＯＹＡ株式会社', 'シチズン時計株式会社', '日本電信電話株式会社', 'ＫＤＤＩ株式会社', 'ソフトバンク株式会社', '株式会社ＮＴＴデータグループ', 'ソフトバンクグループ株式会社', '株式会社しずおかフィナンシャルグループ', '株式会社コンコルディア・フィナンシャルグループ', '株式会社あおぞら銀行', '株式会社三菱ＵＦＪフィナンシャル・グループ', '株式会社りそなホールディングス', '三井住友トラスト・ホールディングス株式会社', '株式会社三井住友フィナンシャルグ

In [16]:
# スクレイピングの結果を格納するための辞書
results = {}

# スクレイピング関数
def fetch_company_data(search_term):
    search_url = 'https://en-hyouban.com/search/'
    params = {'SearchWords': search_term}
    response = requests.get(search_url, params=params)
    if response.ok:
        return response.text
    return None

def find_company_profile_page(html_content, company_name):
    soup = BeautifulSoup(html_content, 'html.parser')
    for a in soup.select('a'):
        if company_name in a.text:
            return urljoin('https://en-hyouban.com', a['href'])
    return None

def fetch_company_profile_page(url):
    response = requests.get(url)
    if response.ok:
        return response.text
    return None

def parse_company_profile_page(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    salary = soup.select_one('.fsize-40.font-change-30.font-roboto.font-weight-bold:nth-of-type(1)').get_text(strip=True) if soup.select_one('.fsize-40.font-change-30.font-roboto.font-weight-bold:nth-of-type(1)') else '情報なし'
    overtime = soup.select_one('.fsize-40.font-change-30.font-roboto.font-weight-bold:nth-of-type(2)').get_text(strip=True) if soup.select_one('.fsize-40.font-change-30.font-roboto.font-weight-bold:nth-of-type(2)') else '情報なし'
    return {
        'salary': salary,
        'overtime': overtime
    }


def parse_company_profile_page(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    salary = soup.select_one('.fsize-40.font-change-30.font-roboto.font-weight-bold:nth-of-type(1)').get_text(strip=True) if soup.select_one('.fsize-40.font-change-30.font-roboto.font-weight-bold:nth-of-type(1)') else '情報なし'
    
    # 使用する警告を修正: 'text' -> 'string'
    overtime_section = soup.find('div', string=lambda t: t and '残業時間' in t)
    if overtime_section:
        overtime = overtime_section.find_next('span', class_='font-change-30').get_text(strip=True) if overtime_section.find_next('span', class_='font-change-30') else '情報なし'
    else:
        overtime = '情報なし'

    return {
        'salary': salary,
        'overtime': overtime
    }


def parse_company_profile_page(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    salary = soup.select_one('.fsize-40.font-change-30.font-roboto.font-weight-bold:nth-of-type(1)').get_text(strip=True) if soup.select_one('.fsize-40.font-change-30.font-roboto.font-weight-bold:nth-of-type(1)') else '情報なし'
    
    # 残業時間情報を含むセクションを探す
    overtime_section = soup.find('div', class_='col-6 px-0 text-center pt-1')
    if overtime_section:
        # 残業時間の値を含むspan要素を探す
        overtime_span = overtime_section.find('span', class_='font-change-30')
        overtime = overtime_span.get_text(strip=True) if overtime_span else '情報なし'
    else:
        overtime = '情報なし'

    return {
        'salary': salary,
        'overtime': overtime
    }


In [17]:
def fetch_salary_details(company_id):
    details_url = f'https://en-hyouban.com/company/{company_id}/salary/'
    response = requests.get(details_url)
    if response.ok:
        return response.text
    return None

def parse_salary_details(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    salary_details = {}
    rows = soup.select('#salary_information_by_age tr')
    for row in rows:
        age_group = row.select_one('.job-type')
        if age_group:
            age_text = age_group.get_text(strip=True)
            # Replace '以上' with '_up' and other age ranges with '_' for consistency
            age_key = age_text.replace('~', '_').replace('歳', '').replace('以上', '_up')
            salary_key = f'salary_{age_key}'  # Create a key like 'salary_25_29'
            salary_value = row.select_one('.salary-font-size').get_text(strip=True) if row.select_one('.salary-font-size') else '情報なし'
            salary_details[salary_key] = salary_value
    return salary_details




def fetch_and_parse_company_data(company):
    search_page_content = fetch_company_data(company)
    if search_page_content:
        profile_page_url = find_company_profile_page(search_page_content, company)
        if profile_page_url:
            sleep(1)  # URLを見つけた後にサーバーの負荷を減らすために待機
            profile_page_content = fetch_company_profile_page(profile_page_url)
            if profile_page_content:
                company_data = parse_company_profile_page(profile_page_content)
                # Extract company ID from profile page URL
                company_id = profile_page_url.split('/')[-2]
                salary_details_content = fetch_salary_details(company_id)
                if salary_details_content:
                    salary_details = parse_salary_details(salary_details_content)
                    company_data.update(salary_details)
                return company, company_data
    return company, None


# 結果をCSVに保存する関数 (更新されたフィールド名)
def save_results_to_csv(results, filename=fr'C:\Users\100ca\Documents\PyCode\scraping\data\results.csv'):
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['company', 'salary', 'overtime', 'salary_25_29', 'salary_30_34', 'salary_35_39', 'salary_40_44', 'salary_45_49', 'salary_50_54', 'salary_55_59', 'salary_60_up']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for company, data in results.items():
            if data:
                writer.writerow({**{'company': company}, **data})

# 並列処理で各企業に対してスクレイピングを実行
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    future_to_company = {executor.submit(fetch_and_parse_company_data, company): company for company in companies}
    results = {}
    for future in concurrent.futures.as_completed(future_to_company):
        company = future_to_company[future]
        try:
            data = future.result()
            results[data[0]] = data[1]
        except Exception as exc:
            print(f'{company} generated an exception: {exc}')

# 結果をCSVに保存
save_results_to_csv(results)


# 結果の出力 (更新されたフィールド名)
for company, data in results.items():
    if data:
        print(f"{company}: 年収={data.get('salary', '情報なし')}, 残業時間={data.get('overtime', '情報なし')}, " +
              f"25-29歳平均年収={data.get('salary_25_29', '情報なし')}, 30-34歳平均年収={data.get('salary_30_34', '情報なし')}, " +
              f"35-39歳平均年収={data.get('salary_35_39', '情報なし')}, 40-44歳平均年収={data.get('salary_40_44', '情報なし')}, " +
              f"45-49歳平均年収={data.get('salary_45_49', '情報なし')}, 50-54歳平均年収={data.get('salary_50_54', '情報なし')}, " +
              f"55-59歳平均年収={data.get('salary_55_59', '情報なし')}, 60歳以上平均年収={data.get('salary_60_up', '情報なし')}")

協和キリン株式会社: 年収=829, 残業時間=27, 25-29歳平均年収=541, 30-34歳平均年収=740, 35-39歳平均年収=744, 40-44歳平均年収=690, 45-49歳平均年収=情報なし, 50-54歳平均年収=1158, 55-59歳平均年収=1316, 60歳以上平均年収=情報なし
アステラス製薬株式会社: 年収=980, 残業時間=26, 25-29歳平均年収=658, 30-34歳平均年収=784, 35-39歳平均年収=831, 40-44歳平均年収=1001, 45-49歳平均年収=1134, 50-54歳平均年収=1383, 55-59歳平均年収=1317, 60歳以上平均年収=情報なし
住友ファーマ株式会社: 年収=720, 残業時間=22, 25-29歳平均年収=577, 30-34歳平均年収=666, 35-39歳平均年収=826, 40-44歳平均年収=863, 45-49歳平均年収=情報なし, 50-54歳平均年収=情報なし, 55-59歳平均年収=情報なし, 60歳以上平均年収=情報なし
武田薬品工業株式会社: 年収=1012, 残業時間=26, 25-29歳平均年収=658, 30-34歳平均年収=787, 35-39歳平均年収=894, 40-44歳平均年収=1056, 45-49歳平均年収=1068, 50-54歳平均年収=1335, 55-59歳平均年収=1567, 60歳以上平均年収=情報なし
塩野義製薬株式会社: 年収=732, 残業時間=27, 25-29歳平均年収=565, 30-34歳平均年収=680, 35-39歳平均年収=744, 40-44歳平均年収=890, 45-49歳平均年収=722, 50-54歳平均年収=情報なし, 55-59歳平均年収=情報なし, 60歳以上平均年収=情報なし
中外製薬株式会社: 年収=1060, 残業時間=28, 25-29歳平均年収=700, 30-34歳平均年収=845, 35-39歳平均年収=996, 40-44歳平均年収=1126, 45-49歳平均年収=1360, 50-54歳平均年収=1312, 55-59歳平均年収=情報なし, 60歳以上平均年収=情報なし
エーザイ株式会社: 年収=861, 残業時間=37, 25-29歳平均年収=673, 30